In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error

# load the data


In [2]:
data = pd.read_json('../../../Downloads/housing.json')


In [11]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit


In [41]:
data = pd.read_json('../../../Downloads/housing.json')

# load the data
data['issue_date'] = pd.to_datetime(data['issue_date'])
# data.set_index('zip_code', inplace=True)


In [46]:
data_subset = data[['zip_code','floor_area_l_a_building_code_definition','valuation','issue_date']]

In [59]:
data_subset['year'] = data_subset['issue_date'].dt.year

/var/folders/n6/7fpdhjgj5nv034gkfyc0mms00000gn/T/ipykernel_15712/1465930034.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset['year'] = data_subset['issue_date'].dt.year


In [86]:
data_subset['persqrt_price'] = data_subset['valuation']/data_subset['floor_area_l_a_building_code_definition']

/var/folders/n6/7fpdhjgj5nv034gkfyc0mms00000gn/T/ipykernel_15712/525483205.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset['persqrt_price'] = data_subset['valuation']/data_subset['floor_area_l_a_building_code_definition']


In [87]:
zip_level_house_data = data_subset.groupby(['zip_code','year']).agg({'persqrt_price':"mean"}).reset_index().dropna()

In [88]:
zip_level_house_data['rank'] = zip_level_house_data.groupby('zip_code')['year'].rank(method='dense', ascending=False)

# filter the dataframe to show only the most recent 5 rows for each zipcode
zip_level_house_data = zip_level_house_data[zip_level_house_data['rank'] <= 5]

# drop the 'rank' column
zip_level_house_data = zip_level_house_data.drop('rank', axis=1)

In [97]:
json_array = zip_level_house_data.to_json(orient='records')


In [100]:
import pickle

In [102]:
crime_data = pd.read_csv("../../../Downloads/crime_date_aligned.csv")

In [105]:
descent_dict = {
    'A': 'Other Asian',
    'B': 'Black',
    'C': 'Chinese',
    'D': 'Cambodian',
    'F': 'Filipino',
    'G': 'Guamanian',
    'H': 'Hispanic/Latin/Mexican',
    'I': 'American Indian/Alaskan Native',
    'J': 'Japanese',
    'K': 'Korean',
    'L': 'Laotian',
    'O': 'Other',
    'P': 'Pacific Islander',
    'S': 'Samoan',
    'U': 'Hawaiian',
    'V': 'Vietnamese',
    'W': 'White',
    'X': 'Unknown',
    'Z': 'Asian Indian'
}


In [108]:
crime_data['ethnicity'] = crime_data['vict_descent'].apply(lambda x: descent_dict.get(x,"Unknown"))

In [110]:
crime_data['age_bucket'] = pd.cut(crime_data['vict_age'], bins=range(0, 101, 10))


In [137]:
sex_dict = {
    'M': 'Male',
    'F': 'Female',
    'X': 'Intersex or unspecified',
    'H': 'Transgender male',
    'N': 'Transgender female'
}

# display the dictionary
print(sex_dict)


{'M': 'Male', 'F': 'Female', 'X': 'Intersex or unspecified', 'H': 'Transgender male', 'N': 'Transgender female'}


In [135]:
crime_data['vict_sex'].unique()

array(['M', 'F', 'X', 'H', 'N'], dtype=object)

In [138]:
sex_dict['M']

'Male'

In [140]:
def gender_extract(x):
    try:
        return sex_dict[x]
    except:
        print(x)

In [141]:
crime_data['Victim_gender'] = crime_data['vict_sex'].apply(gender_extract)

In [143]:
crime_data['Victim_gender'].unique()

array(['Male', 'Female', 'Intersex or unspecified', 'Transgender male',
       'Transgender female'], dtype=object)

In [144]:
crime_data['date_occ'] = pd.to_datetime(crime_data['date_occ'])


In [145]:
crime_data['date_occ_year'] = crime_data['date_occ'].dt.year

In [146]:
crime_data.columns

Index(['Unnamed: 0', 'dr_no', 'date_rptd', 'date_occ', 'area', 'area_name',
       'rpt_dist_no', 'crm_cd', 'crm_cd_desc', 'vict_age', 'vict_sex',
       'vict_descent', 'vict_descent.1', 'location', 'lat', 'lon', 'zip_code',
       'ethnicity', 'age_bucket', 'Victim_gender', 'date_occ_year'],
      dtype='object')

In [190]:
crime_data_crimtype = crime_data.groupby(['zip_code','date_occ_year','crm_cd_desc']).agg({'area_name':'count'}).reset_index()



In [200]:
crime_data_crimtype['rank'] = crime_data_crimtype.groupby(['zip_code','date_occ_year'])['area_name'].rank(method='dense', ascending=False)


In [201]:
# filter the dataframe to show only the most recent 5 rows for each zipcode
crime_data_crimtype = crime_data_crimtype[crime_data_crimtype['rank'] <= 5]


In [203]:



# drop the 'rank' column
crime_data_crimtype = crime_data_crimtype.drop('rank', axis=1)

In [147]:


crime_data_ethnicity = crime_data.groupby(['zip_code','date_occ_year','ethnicity']).agg({'area_name':'count'}).reset_index()



crime_data_age_bucket = crime_data.groupby(['zip_code','date_occ_year','age_bucket']).agg({'area_name':'count'}).reset_index()


crime_data_Victim_gender = crime_data.groupby(['zip_code','date_occ_year','Victim_gender']).agg({'area_name':'count'}).reset_index()



In [162]:
crime_data_age_bucket

,zip_code,date_occ_year,age_bucket,area_name
0,90001,2013,"(0, 10]",1
1,90001,2013,"(10, 20]",13
2,90001,2013,"(20, 30]",15
3,90001,2013,"(30, 40]",18
4,90001,2013,"(40, 50]",10
...,...,...,...,...
19695,97003,2022,"(50, 60]",0
19696,97003,2022,"(60, 70]",0
19697,97003,2022,"(70, 80]",0
19698,97003,2022,"(80, 90]",0


In [204]:
import pandas as pd

# create sample dataframes
# crime_data_ethnicity = pd.DataFrame({
#     'zip_code': [90001, 90001],
#     'date_occ_year': [2013, 2013],
#     'ethnicity': ['Black', 'Hispanic/Latin/Mexican'],
#     'area_name': [23, 49]
# })

# crime_data_age_bucket = pd.DataFrame({
#     'zip_code': [90001, 90001],
#     'date_occ_year': [2013, 2013],
#     'age_bucket': ['(0, 10]', '(10, 20]'],
#     'area_name': [1, 13]
# })

# crime_data_Victim_gender = pd.DataFrame({
#     'zip_code': [90001, 90001],
#     'date_occ_year': [2013, 2013],
#     'Victim_gender': ['Filipino', 'Intersex or unspecified'],
#     'area_name': [36, 43]
# })

# create an empty dictionary to store the results
result_dict = {}

# iterate over each zip_code
for zip_code in set(crime_data_ethnicity['zip_code']):
    # filter the dataframes for the current zip_code
    ethnicity_data = crime_data_ethnicity[crime_data_ethnicity['zip_code'] == zip_code]
    age_data = crime_data_age_bucket[crime_data_age_bucket['zip_code'] == zip_code]
    gender_data = crime_data_Victim_gender[crime_data_Victim_gender['zip_code'] == zip_code]
    crime_typedt = crime_data_crimtype[crime_data_crimtype['zip_code']==zip_code]
    
    # create an empty dictionary to store the nested structures for each year
    year_dict = {}
    
    # iterate over each year for the current zip_code
    for year in set(ethnicity_data['date_occ_year']):
        # filter the dataframes for the current year
        ethnicity_year_data = ethnicity_data[ethnicity_data['date_occ_year'] == year]
        age_year_data = age_data[age_data['date_occ_year'] == year]
        gender_year_data = gender_data[gender_data['date_occ_year'] == year]
        crim_year_data = crime_typedt[crime_typedt['date_occ_year'] == year]
       
    
    
        crime_dict = dict(zip(crim_year_data['crm_cd_desc'], crim_year_data['area_name']))
        print(crime_dict)
                
        
        # create nested structures for gender, sex, and ethnicity distribution
        gender_dict = dict(zip(gender_year_data['Victim_gender'], gender_year_data['area_name']))
        lst = []
        for interval in age_year_data['age_bucket'].values:
            tuple_interval = (interval.left, interval.right)
            lst.append(tuple_interval)
        sex_dict_1 =  dict(zip(lst, age_year_data['area_name']))
        ethnicity_dict = dict(zip(ethnicity_year_data['ethnicity'], ethnicity_year_data['area_name']))
        
#         # create nested structure for sex distribution by iterating over age buckets
#         for age_bucket in set(age_year_data['age_bucket']):
#             age_bucket_data = age_year_data[age_year_data['age_bucket'] == age_bucket]
#             sex_dict_1[age_bucket] = dict(zip(age_bucket_data['area_name'], age_bucket_data['area_name']))
        
        # combine the nested structures for gender, sex, and ethnicity distribution
        year_dict[year] = {
            'gender_distribution': gender_dict,
            'age_distribution': sex_dict_1,
            'ethnicity_distribution': ethnicity_dict,
            'top5_crimes':crime_dict
        }
    
    # add the nested structures for the current zip_code to the result dictionary
    result_dict[zip_code] = year_dict

# convert the result dictionary to an array of nested jsons
result_array = []
for key, value in result_dict.items():
    result_array.append({'zip_code': key, 'year_data': value})

# display the result array
# print(result_array)


{'SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH': 1}
{'BATTERY - SIMPLE ASSAULT': 1, 'THEFT OF IDENTITY': 1}
{'BURGLARY FROM VEHICLE, ATTEMPTED': 1}
{'BURGLARY FROM VEHICLE': 49, 'THEFT PLAIN - PETTY ($950 & UNDER)': 9, 'THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)': 8, 'VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)': 7, 'BATTERY - SIMPLE ASSAULT': 6, 'THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD': 6}
{'BURGLARY FROM VEHICLE': 43, 'THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD': 7, 'VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)': 7, 'THEFT PLAIN - PETTY ($950 & UNDER)': 6, 'VANDALISM - MISDEAMEANOR ($399 OR UNDER)': 5, 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT': 4, 'BATTERY - SIMPLE ASSAULT': 4}
{'BURGLARY FROM VEHICLE': 32, 'THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)': 5, 'BATTERY - SIMPLE ASSAULT': 3, 'THEFT PLAIN - PETTY ($950 & UNDER)': 3, 'VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)': 3, 'ASSAULT WITH D

In [207]:


with open("result_array_crime_distribution.pkl",'wb') as topic_out:
    pickle.dump(result_array,topic_out)

In [130]:
crime_data_Victim_gender.head(2)

,zip_code,date_occ_year,Victim_gender,area_name
0,90001,2013,Filipino,36
1,90001,2013,Intersex or unspecified,43


In [127]:
crime_data_ethnicity

,zip_code,date_occ_year,ethnicity,area_name
0,90001,2013,Black,23
1,90001,2013,Hispanic/Latin/Mexican,49
2,90001,2013,Other,3
3,90001,2013,Other Asian,2
4,90001,2013,White,2
...,...,...,...,...
12791,97003,2021,Other,130
12792,97003,2021,Other Asian,17
12793,97003,2021,Unknown,222
12794,97003,2021,Vietnamese,1


In [115]:
crime_data.groupby(['vict_sex']).count()

,Unnamed: 0,dr_no,date_rptd,date_occ,area,area_name,rpt_dist_no,crm_cd,crm_cd_desc,vict_age,vict_descent,vict_descent.1,location,lat,lon,zip_code,ethnicity,age_bucket
vict_sex,,,,,,,,,,,,,,,,,,
F,860883,860883,860883,860883,860883,860883,860883,860883,860883,860883,860883,860883,860883,860883,860883,860883,860883,849527
H,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,126
M,954545,954545,954545,954545,954545,954545,954545,954545,954545,954545,954545,954545,954545,954545,954545,954545,954545,860070
N,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,16
X,105419,105419,105419,105419,105419,105419,105419,105419,105419,105419,105419,105419,105419,105419,105419,105419,105419,10982


In [112]:
crime_data.vict_sex.unique()

array(['M', 'F', 'X', 'H', 'N'], dtype=object)

In [101]:
with open("Housing_json_array.pkl",'wb') as node_similarity_file:
    pickle.dump(json_array,node_similarity_file)

In [96]:
zip_level_house_data.reset_index(drop=True)

,zip_code,year,persqrt_price
0,90001.0,2016,101.404398
1,90001.0,2018,98.401430
2,90001.0,2020,96.069824
3,90001.0,2021,99.027207
4,90001.0,2022,121.721081
...,...,...,...
456,91607.0,2019,108.618485
457,91607.0,2020,84.117501
458,91607.0,2021,115.124652
459,91607.0,2022,113.771828


In [82]:
zip_level_house_data = zip_level_house_data.sort_values(['zip_code', 'year'], ascending=[True, False])


In [84]:

# group the dataframe by zipcode and get the 5 largest values for each group based on the year column
zip_level_house_data = zip_level_house_data.groupby('zip_code').apply(lambda x: x.nlargest(5, 'year'))

# reset the index and drop the 'level_1' column
# zip_level_house_data = zip_level_house_data.reset_index(drop=True)

In [85]:
zip_level_house_data

zip_code  year  persqrt_price
zip_code                                    
90001.0  9      90001.0  2022     121.721081
         8      90001.0  2021      99.027207
         7      90001.0  2020      96.069824
         5      90001.0  2018      98.401430
         3      90001.0  2016     101.404398
...                 ...   ...            ...
91607.0  1091   91607.0  2023      86.388942
         1090   91607.0  2022     113.771828
         1089   91607.0  2021     115.124652
         1088   91607.0  2020      84.117501
         1087   91607.0  2019     108.618485

[461 rows x 3 columns]

In [53]:
data_subset.year.max()

Timestamp('2021-09-30 00:00:00')

In [47]:
data_subset.zip_code.nunique()

116

In [43]:
data.columns

Index(['zip_code', 'address_end', 'floor_area_l_a_building_code_definition',
       'work_description', ':@computed_region_qz3q_ghft',
       ':@computed_region_k96s_3jcv', 'reference_old_permit',
       ':@computed_region_kqwf_mjcx', 'census_tract', 'permit_category',
       'initiating_office', ':@computed_region_2dna_qi2s', 'assessor_parcel',
       'applicant_first_name', 'zone', 'assessor_book',
       'license_expiration_date', ':@computed_region_tatf_ua23',
       'license_type', 'valuation', 'of_residential_dwelling_units',
       'pcis_permit', 'address_start', ':@computed_region_ur2y_g4cx',
       'street_name', 'street_suffix', 'street_direction',
       'applicant_last_name', 'license',
       'floor_area_l_a_zoning_code_definition', 'issue_date', 'assessor_page',
       'contractors_business_name', 'tract', 'lot', 'permit_sub_type',
       'location_1', 'of_stories', 'permit_type', 'principal_first_name',
       'block', 'contractor_state', 'principal_middle_name', 'contra

In [21]:
data['valuation'] = data.groupby('zip_code')['valuation'].transform(lambda x: x.fillna(x.mean()))


In [22]:

# generate monthly prices at zip code level
monthly_prices = data.groupby(['zip_code', pd.Grouper(freq='M')])['valuation'].mean().reset_index()
monthly_prices.set_index('issue_date', inplace=True)

# create lagged features
monthly_prices['price_lag1'] = monthly_prices.groupby('zip_code')['valuation'].shift(1)
monthly_prices['price_lag3'] = monthly_prices.groupby('zip_code')['valuation'].shift(3)
monthly_prices['price_lag12'] = monthly_prices.groupby('zip_code')['valuation'].shift(12)


In [29]:
len(train_X)

1219

In [28]:
test_X.isna().sum()

price_lag1      14
price_lag3      42
price_lag12    168
dtype: int64

In [35]:
monthly_prices.dropna(inplace=True)

In [37]:

# split the data into training and testing sets using time series split
tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(monthly_prices):
    train = monthly_prices.iloc[train_index]
    test = monthly_prices.iloc[test_index]
    # preprocess the data
    train = train.dropna()
    test = test.dropna()
    # preprocess the data
    train_X, train_y = train[['price_lag1', 'price_lag3', 'price_lag12']], train['valuation']
    test_X, test_y = test[['price_lag1', 'price_lag3', 'price_lag12']], test['valuation']

    # fit the model
    model = LinearRegression()
    model.fit(train_X, train_y)

    # make predictions
    y_pred = model.predict(test_X)

    # evaluate the model
    mse = mean_squared_error(test_y, y_pred)
    rmse = np.sqrt(mse)
    print(f"RMSE: {np.log10(rmse)}")

# forecast prices for the next month
last_month = monthly_prices.groupby('zip_code').tail(1)
future_X = last_month[['price_lag1', 'price_lag3', 'price_lag12']].values
future_pred = model.predict(future_X)
print(f"Forecasted price: {future_pred}")


RMSE: 6.880240614240576
RMSE: 6.55439686649853
RMSE: 6.523116592943083
RMSE: 6.7433682578591885
RMSE: 6.398316965761282
Forecasted price: [ 1424397.21869372  1368369.22874744  1566636.36140708  1770936.1350838
  4406248.40260489  2099233.57829905  2430011.22190112  2953062.06824349
 34596785.23104956  1458652.42190683  6215297.49652386  3930582.34571658
  4130910.63421382 31834878.18121794  1891585.71136295  9454271.49032831
  1882499.86148988  1632195.34596538  3041720.57061527  1313758.74344883
  1824789.42464669  2434149.18520604  2550046.83485367  1688953.08256417
  1388515.30644809  1321792.3952293   8183829.05572454  1345463.22102592
  1648981.02369766  1884952.73755495  1765725.69979658  1626910.81764572
  1424040.70061706  1421065.43409088  1350438.29156206  2835784.52010514
  1382631.59349651  2744814.18338559  1342530.96404777  1873734.65853935
  2028425.66795703  2759138.97716922  1506384.80717988  1886273.57457984
  3234242.40225136  1388611.29212204  1383852.20049821  1503

/Users/satwant/Documents/pyenv/env/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
